In [1]:
import pandas as pd
import json
from functools import reduce
from collections import defaultdict

In [2]:
NLP_RESULTS = '/Users/nathansuberi/Documents/GitHub/ResourceWatchCode/Analytic Tools/climatewatch_ndc_nlp/data/nlp_results.txt'
COUNTRY_CATEGORIES = '/Users/nathansuberi/Documents/GitHub/ResourceWatchCode/Analytic Tools/climatewatch_ndc_nlp/data/country_grouping.csv'

In [3]:
categories = pd.read_csv(COUNTRY_CATEGORIES).set_index('countryCode')
with open(NLP_RESULTS, 'r') as src:
    nlp_results = json.load(src)

In [4]:
nlp_results

{'AFG': {'access': 3,
  'accsap': 1,
  'achievements': 1,
  'action plans': 1,
  'activities': 1,
  'adaptation': 4,
  'adaptation measures': 1,
  'adaptation needs': 3,
  'adaptation technology transfer': 1,
  'adaptation undertakings': 1,
  'adaptation visions': 1,
  'adaptive': 1,
  'addition': 1,
  'adoption': 2,
  'advocates': 1,
  'afforestation': 1,
  'afghanistan ccsap': 1,
  'afghanistan greenhouse gas inventory report': 1,
  'afghanistan national renewable energy policy': 1,
  'afghanistan practitioners': 1,
  'afghanistan vocational': 1,
  'agencies': 1,
  'agriculture': 11,
  'agriculture sector': 1,
  'alternative': 1,
  'altitudes': 1,
  'analysis': 1,
  'analysts': 1,
  'ands': 1,
  'anrep': 1,
  'appliances': 1,
  'approaches': 1,
  'area': 2,
  'arrangements': 1,
  'assistance': 1,
  'attention': 1,
  'availability': 1,
  'awareness': 2,
  'barriers': 2,
  'bau': 1,
  'biodegradable waste': 1,
  'biodiversity': 1,
  'biomass': 1,
  'budget': 1,
  'building codes': 1,
 

In [ ]:
categories = categories.drop('countryName', axis=1)

In [10]:
def create_summaries(agg, cntry, nlp, group, groupings):
    try:
        cntry_group = groupings.loc[cntry, group]
        for word, count in nlp[cntry].items():
            agg[cntry_group][word] += count
    except:
        pass
    return agg
    
def create_counts(agg, cntry_nlp):
    try:
        for word in cntry_nlp:
            agg[word] += 1
    except:
        pass
    return agg
    
def dd_wrapper():
    return defaultdict(int)
    
def flatten(dd):
    dd = dict(dd)
    for key, _dd in dd.items():
        dd[key] = dict(_dd)
    return dd
        

In [16]:
word_counts = dict(reduce(lambda agg, cntry_nlp: create_counts(agg, 
                                               cntry_nlp[1]), 
                                          nlp_results.items(), 
                                          defaultdict(int)))

In [17]:
word_counts

{'access': 93,
 'accordance': 86,
 'account': 73,
 'accumulation rate': 1,
 'act': 5,
 'action plan': 32,
 'actions standard': 1,
 'activities': 104,
 'adaptation': 123,
 'adaptation actions': 64,
 'adaptation plan': 9,
 'adaptation strategies': 21,
 'addition': 99,
 'advantage': 11,
 'agencies': 44,
 'agriculture': 137,
 'aichi target': 1,
 'aim': 31,
 'airport': 3,
 'ambition grenada': 1,
 'americas caribbean initiative': 1,
 'analysis': 68,
 'approach': 95,
 'approval': 19,
 'aquifers': 14,
 'area': 86,
 'arrangements': 35,
 'assessments': 39,
 'assistance': 54,
 'atmosphere': 43,
 'awareness': 78,
 'backing': 2,
 'bamboo': 1,
 'bamboos': 2,
 'bau': 75,
 'beach monitoring program': 1,
 'beaches': 13,
 'benefits': 58,
 'biofuel blends': 1,
 'biological diversity': 1,
 'blend': 3,
 'blow': 1,
 'blue economy initiative': 1,
 'building sectors': 1,
 'buildings': 61,
 'businesses': 11,
 'cabinet': 18,
 'calculations': 19,
 'capacities': 58,
 'capacity': 110,
 'capacity building': 92,
 'c

In [7]:
topics_by_income = flatten(reduce(lambda agg, country: create_summaries(agg, 
                                               country, 
                                               nlp_results, 
                                               'byIncomeTXT', 
                                               categories), 
                                          nlp_results, 
                                          defaultdict(dd_wrapper)))

In [ ]:
grouped_results = {}
for cat in categories.columns:
    grouped_results[cat] = flatten(reduce(lambda agg, country: create_summaries(agg, 
                                               country, 
                                               nlp_results, 
                                               cat, 
                                               categories), 
                                          nlp_results, 
                                          defaultdict(dd_wrapper)))

In [ ]:
with open('grouped_results.txt', 'w') as dst:
    dst.write(json.dumps(grouped_results))

In [29]:
import operator
with open('word_counts_by_num_docs.txt', 'w') as dst:
    dst.write(json.dumps(sorted(word_counts.items(), key=operator.itemgetter(1), reverse=True)))